In [ ]:
#input
import pydicom as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import sklearn.preprocessing as pp
import skimage.filters as filters
import skimage.feature as feature
import skimage.restoration as restoration
import skimage.exposure as exposure
import os

In [ ]:
work_dir = './Data/Raw'
dcm_files = [pd.read_file(os.path.join(work_dir,file_path)) for file_path in os.listdir(work_dir)]

In [ ]:
#cropping by metadata

#input: img - dcm file
#output: cropped pixel array

def metacrop(file):
    for key in file.dir():
        value = getattr(file, key, "")
        if(type(value) == pd.sequence.Sequence and key == "SequenceOfUltrasoundRegions"):
            value = value[0]
            break
    x0, x1, y0, y1 = None, None, None, None
    for key in value.dir():
        if key == "RegionLocationMinX0":
            x0 = getattr(value, key, "")
        if key == "RegionLocationMaxX1":
            x1 = getattr(value, key, "")
        if key == "RegionLocationMinY0":
            y0 = getattr(value, key, "")
        if key == "RegionLocationMaxY1":
            y1 = getattr(value, key, "")
    print(x0, x1, y0, y1)
    if(x0 == 0):
        return colorcrop2(file.pixel_array)
    return file.pixel_array[y0:y1,x0:x1]

In [ ]:
def colorcrop(pixel_arr):
    midCol = pixel_arr.shape[1]//2
    print(midCol)

    # Left Side
    flag = False
    ind = -1
    for i in range(midCol, 0, -1):
        colArr = pixel_arr[:,i,0]
        if(np.mean(colArr) == 0):
            flag = True
            ind = i
            break
    flag2 = False
    ind2 = -1
    for i in range(midCol, pixel_arr.shape[1]):
        colArr = pixel_arr[:,i,0]
        if(np.mean(colArr) == 0):
            flag2 = True
            ind2 = i
            break
    if(flag and flag2):
        return pixel_arr[:,ind:ind2]
    return pixel_arr

In [ ]:
#cropping by x-axis black border

#input: pixel array
#output: cropped pixel array
def colorcrop2(pixel_arr):
    midCol = pixel_arr.shape[1]//2
    x = pixel_arr[:,midCol:].sum(axis=0)[:,0]
    distance = np.where(x == 0)[0][0]
    ind = midCol - distance
    ind2 = midCol + distance
    print(ind, ind2)
    return pixel_arr[:,ind:ind2]

In [ ]:
#Input a image as a NumpyArray
#It will output a flipped image
def flip(img):
    return np.fliplr(img)

In [ ]:
#Gaussian

'''input: img = np array containing pixels of image to be processed
         sig: double value for setting the sigma blurring filter
  output: smooth_img = np array smoothed using gaussian filter'''

def gaussian_blur(img, sig):
    smooth_img = filters.gaussian(img, sigma = sig)
    return smooth_img

In [ ]:
def sobel(img):
    return filters.sobel(img)

In [ ]:
def canny(img, sigma):
    return feature.canny(img, sigma = sigma)

In [ ]:
#wavelet denoising

'''input: img = np array containing pixels of image to be processed
  output: denoise_img = np array with less noise using wavelet denoising of type db1'''

def denoising(img):
    denoise_img = restoration.denoise_wavelet(img)
    return denoise_img

In [ ]:
def clahe(img):
    return exposure.equalize_adapthist(img)

In [ ]:
def conv_2d_3d(img):
    arr = np.array(np.ones((img.shape[0], img.shape[1], 3), dtype=float))
    print(arr.shape)
    arr[:,:,0] = img
    arr[:,:,1] = img
    arr[:,:,2] = img
    return arr

In [ ]:
folder_path = "./Data/Reg"
for counter, dcm in enumerate(dcm_files):
    metacrop_arr = metacrop(dcm)
    blur_arr = gaussian_blur(metacrop_arr, 15)
    clahe_arr = clahe(blur_arr)
    #canny_arr_2d = canny(clahe_arr[:,:,0], 1.5)
    #canny_arr = conv_2d_3d(canny_arr_2d)
    #denoise_arr = denoising(clahe_arr)
    #flip_edge_arr = flip(canny_arr)
    flip_arr = flip(clahe_arr)
    file_path = os.path.join(folder_path, str(counter))
    np.save(file_path+".npy", clahe_arr)
    np.save(file_path+"rev.npy", flip_arr)
    #np.save(file_path+"edge.npy", canny_arr)
    #np.save(file_path+"edgerev.npy", flip_edge_arr)

In [ ]:
plt.imshow(dcm.pixel_array)
plt.show()
plt.imshow(metacrop_arr)
plt.show()
plt.imshow(blur_arr)
plt.show()
plt.imshow(clahe_arr)
plt.show()
plt.imshow(canny_arr)
plt.show()
#plt.imshow(denoise_arr)
#plt.show()
plt.imshow(flip_arr)
plt.show()